# Q1. Create a table called employees with the following structure?
- emp_id (integer, should not be NULL and should be a primary key)Q
- emp_name (text, should not be NULL)Q
- age (integer, should have a check constraint to ensure the age is at least 18)Q
- email (text, should be unique for each employee)Q
- salary (decimal, with a default value of 30,000).

# Write the SQL query to create the above table with all constraints.

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY NOT NULL,
    emp_name TEXT NOT NULL,
    age INTEGER NOT NULL CHECK (age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL DEFAULT 30000
);
""")

print("Table 'employees' created successfully.")

cursor.execute("PRAGMA table_info(employees);")
for column in cursor.fetchall():
    print(column)


Table 'employees' created successfully.
(0, 'emp_id', 'INTEGER', 1, None, 1)
(1, 'emp_name', 'TEXT', 1, None, 0)
(2, 'age', 'INTEGER', 1, None, 0)
(3, 'email', 'TEXT', 0, None, 0)
(4, 'salary', 'DECIMAL', 0, '30000', 0)


# Q2. Explain the purpose of constraints and how they help maintain data integrity in a database. Provide examples of common types of constraints.

### Purpose of Constraints in a Database

**Constraints** are rules enforced on data in database tables. Their main purpose is to **maintain data integrity**, **ensure accuracy**, and **enforce business logic** by restricting the type of data that can be inserted, updated, or deleted.

They help:
- Prevent invalid data entry.
- Maintain consistency across rows and tables.
- Enforce unique identification.
- Enforce valid relationships between tables.

---

###  Common Types of Constraints (with Examples)

| Constraint Type | Purpose                                           | Example                                 |
|------------------|---------------------------------------------------|------------------------------------------|
| **PRIMARY KEY**   | Uniquely identifies each record; not NULL         | `emp_id INTEGER PRIMARY KEY`             |
| **NOT NULL**      | Ensures a column cannot have `NULL` values        | `emp_name TEXT NOT NULL`                 |
| **UNIQUE**        | Ensures all values in a column are different      | `email TEXT UNIQUE`                      |
| **CHECK**         | Ensures values meet a specific condition          | `age INTEGER CHECK(age >= 18)`           |
| **DEFAULT**       | Assigns a default value if none is specified      | `salary DECIMAL DEFAULT 30000`           |
| **FOREIGN KEY**   | Links to a primary key in another table           | `dept_id INTEGER REFERENCES departments(dept_id)` |

---

###  Benefits for Data Integrity

- Prevents duplicate or orphan records.
- Ensures required fields are filled.
- Enforces valid data ranges and formats.
- Supports reliable relationships between tables.

# Q3. Why would you apply the NOT NULL constraint to a column? Can a primary key contain NULL values? Justify your answer.

The `NOT NULL` constraint is used to ensure that a column cannot have missing (NULL) values. This is important when the data in that column is essential for the validity or operation of the database.

#### Reasons to Use `NOT NULL`:
- Ensures important fields are always filled in (e.g., names, dates, IDs).
- Prevents errors in queries and calculations due to missing values.
- Maintains consistency and reliability of the data.

**Example**:
```sql
emp_name TEXT NOT NULL


### Can a Primary Key Contain `NULL` Values?

No, a primary key cannot contain `NULL` values.

#### Justification:
- A primary key uniquely identifies each record in a table.
- `NULL` represents an unknown or missing value and is not considered equal to any other value, including another `NULL`.
- If `NULL` were allowed in a primary key, it would break the rule of uniqueness required by primary keys.
- In SQL, defining a column as a `PRIMARY KEY` automatically applies both `NOT NULL` and `UNIQUE` constraints.

#### Example:
```sql
emp_id INTEGER PRIMARY KEY


# Q4.  Explain the steps and SQL commands used to add or remove constraints on an existing table. Provide an example for both adding and removing a constraint.

### Modifying Constraints on an Existing Table in SQL

In SQL, you can modify constraints on an existing table using the `ALTER TABLE` statement. However, the exact syntax and support for modifying constraints vary slightly across different database systems (e.g., MySQL, PostgreSQL, SQLite). Below are general steps and examples for **adding** and **removing** constraints.

---

### 1. Adding a Constraint

Use the `ALTER TABLE` command along with `ADD CONSTRAINT` to add a new constraint.

#### Syntax:
```sql
ALTER TABLE table_name
ADD CONSTRAINT constraint_name constraint_type (column_name);


### 2. Removing a Constraint
Use the ALTER TABLE command along with DROP CONSTRAINT to remove an existing constraint.

#### Syntax:
```sql

ALTER TABLE table_name
DROP CONSTRAINT constraint_name;


In [ ]:
import sqlite3

# Step 1: Connect to SQLite in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Step 2: Create the original table without constraints on 'email'
cursor.execute("""
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    emp_name TEXT NOT NULL,
    age INTEGER NOT NULL CHECK(age >= 18),
    email TEXT,
    salary DECIMAL DEFAULT 30000
);
""")
print("Original table created.")

# Step 3: Recreate the table with a UNIQUE constraint on 'email'
cursor.execute("""
CREATE TABLE employees_new (
    emp_id INTEGER PRIMARY KEY,
    emp_name TEXT NOT NULL,
    age INTEGER NOT NULL CHECK(age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL DEFAULT 30000
);
""")

# Step 4: Copy data (none in this case, but shown for completeness)
cursor.execute("INSERT INTO employees_new SELECT * FROM employees;")

# Step 5: Drop old table and rename new one
cursor.execute("DROP TABLE employees;")
cursor.execute("ALTER TABLE employees_new RENAME TO employees;")
print("UNIQUE constraint added to 'email' column.")

# Step 6: Drop UNIQUE constraint by recreating table without it
cursor.execute("""
CREATE TABLE employees_temp (
    emp_id INTEGER PRIMARY KEY,
    emp_name TEXT NOT NULL,
    age INTEGER NOT NULL CHECK(age >= 18),
    email TEXT,
    salary DECIMAL DEFAULT 30000
);
""")

cursor.execute("INSERT INTO employees_temp SELECT * FROM employees;")
cursor.execute("DROP TABLE employees;")
cursor.execute("ALTER TABLE employees_temp RENAME TO employees;")
print("UNIQUE constraint removed from 'email' column.")


Original table created.
UNIQUE constraint added to 'email' column.
UNIQUE constraint removed from 'email' column.


# Q5. Explain the consequences of attempting to insert, update, or delete data in a way that violates constraints. Provide an example of an error message that might occur when violating a constraint.

### Consequences of Violating Constraints in SQL

When you attempt to **insert**, **update**, or **delete** data in a way that violates a constraint, the database system will **reject the operation** and return an **error message**. Constraints are designed to enforce data integrity and prevent invalid data from being stored in the database.

---

### Types of Violations and Their Consequences

#### 1. `NOT NULL` Violation
- **Action**: Inserting or updating a column with a `NULL` value where `NOT NULL` is enforced.
- **Consequence**: Operation fails.
- **Example Error**: NOT NULL constraint failed: employees.emp_name

#### 2. `UNIQUE` Violation
- **Action**: Inserting or updating a value that duplicates an existing value in a column with a `UNIQUE` constraint.
- **Consequence**: Operation fails to prevent duplicate entries.
- **Example Error**: UNIQUE constraint failed: employees.email

#### 3. `CHECK` Constraint Violation
- **Action**: Inserting or updating data that violates the condition defined in a `CHECK` constraint.
- **Consequence**: Operation is blocked.
- **Example Error**: CHECK constraint failed: employees

#### 4. `PRIMARY KEY` Violation
- **Action**: Inserting a duplicate or `NULL` value into a `PRIMARY KEY` column.
- **Consequence**: Operation fails since primary keys must be unique and not null.
- **Example Error**: UNIQUE constraint failed: employees.emp_id

#### 5. `FOREIGN KEY` Violation
- **Action**: Inserting a value that does not exist in the referenced table or deleting a referenced record.
- **Consequence**: Prevents orphan records and maintains referential integrity.
- **Example Error**: FOREIGN KEY constraint failed


# Q6. You created a products table without constraints as follows:

CREATE TABLE products (

    product_id INT,

    product_name VARCHAR(50),

    price DECIMAL(10, 2));
# Now, you realise that?
# - The product_id should be a primary keyQ
# - The price should have a default value of 50.00

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE products (
    product_id INT,
    product_name VARCHAR(50),
    price DECIMAL(10, 2)
);
""")

cursor.execute("INSERT INTO products (product_id, product_name, price) VALUES (1, 'Notebook', 80.00);")
cursor.execute("INSERT INTO products (product_id, product_name) VALUES (2, 'Pen');")

cursor.execute("""
CREATE TABLE products_new (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50),
    price DECIMAL(10, 2) DEFAULT 50.00
);
""")

cursor.execute("""
INSERT INTO products_new (product_id, product_name, price)
SELECT product_id, product_name,
       COALESCE(price, 50.00)
FROM products;
""")

cursor.execute("DROP TABLE products;")
cursor.execute("ALTER TABLE products_new RENAME TO products;")

cursor.execute("PRAGMA table_info(products);")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM products;")
for row in cursor.fetchall():
    print(row)


(0, 'product_id', 'INT', 0, None, 1)
(1, 'product_name', 'VARCHAR(50)', 0, None, 0)
(2, 'price', 'DECIMAL(10, 2)', 0, '50.00', 0)
(1, 'Notebook', 80)
(2, 'Pen', 50)


# Q7. You have two tables:
Write a query to fetch the student_name and class_name for each student using an INNER JOIN.

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE Students (
    student_id INTEGER,
    student_name TEXT,
    class_id INTEGER
);
""")

cursor.execute("""
CREATE TABLE Classes (
    class_id INTEGER,
    class_name TEXT
);
""")

cursor.executemany("""
INSERT INTO Students (student_id, student_name, class_id)
VALUES (?, ?, ?);
""", [
    (1, 'Alice', 101),
    (2, 'Bob', 102),
    (3, 'Charlie', 101)
])

cursor.executemany("""
INSERT INTO Classes (class_id, class_name)
VALUES (?, ?);
""", [
    (101, 'Math'),
    (102, 'Science'),
    (103, 'History')
])

cursor.execute("""
SELECT s.student_name, c.class_name
FROM Students s
INNER JOIN Classes c
ON s.class_id = c.class_id;
""")

results = cursor.fetchall()
for row in results:
    print(row)


('Alice', 'Math')
('Bob', 'Science')
('Charlie', 'Math')


# Q8. Consider the following three tables:
Write a query that shows all order_id, customer_name, and product_name, ensuring that all products are listed even if they are not associated with an order

Hint: (use INNER JOIN and LEFT JOIN)5

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE Orders (
    order_id INTEGER,
    order_date TEXT,
    customer_id INTEGER
);
""")

cursor.execute("""
CREATE TABLE Customers (
    customer_id INTEGER,
    customer_name TEXT
);
""")

cursor.execute("""
CREATE TABLE Products (
    product_id INTEGER,
    product_name TEXT,
    order_id INTEGER
);
""")

cursor.executemany("INSERT INTO Orders VALUES (?, ?, ?);", [
    (1, '2024-01-01', 101),
    (2, '2024-01-03', 102)
])

cursor.executemany("INSERT INTO Customers VALUES (?, ?);", [
    (101, 'Alice'),
    (102, 'Bob')
])

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?);", [
    (1, 'Laptop', 1),
    (2, 'Phone', None)
])

cursor.execute("""
SELECT
    o.order_id,
    c.customer_name,
    p.product_name
FROM Products p
LEFT JOIN Orders o ON p.order_id = o.order_id
LEFT JOIN Customers c ON o.customer_id = c.customer_id;
""")

for row in cursor.fetchall():
    print(row)


(1, 'Alice', 'Laptop')
(None, None, 'Phone')


# Q9. Given the following tables:
Write a query to find the total sales amount for each product using an INNER JOIN and the SUM() function.

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("CREATE TABLE Sales (sale_id INTEGER, product_id INTEGER, amount INTEGER);")
cursor.execute("CREATE TABLE Products (product_id INTEGER, product_name TEXT);")

cursor.executemany("INSERT INTO Sales VALUES (?, ?, ?);", [
    (1, 101, 500),
    (2, 102, 300),
    (3, 101, 700)
])

cursor.executemany("INSERT INTO Products VALUES (?, ?);", [
    (101, 'Laptop'),
    (102, 'Phone')
])

cursor.execute("""
SELECT
    p.product_name,
    SUM(s.amount) AS total_sales
FROM Sales s
INNER JOIN Products p ON s.product_id = p.product_id
GROUP BY p.product_name;
""")

for row in cursor.fetchall():
    print(row)


('Laptop', 1200)
('Phone', 300)


# Q10. You are given three tables:
Write a query to display the order_id, customer_name, and the quantity of products ordered by each customer using an INNER JOIN between all three tables

In [ ]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute("CREATE TABLE Orders (order_id INTEGER, order_date TEXT, customer_id INTEGER);")
cursor.execute("CREATE TABLE Customers (customer_id INTEGER, customer_name TEXT);")
cursor.execute("CREATE TABLE Order_Details (order_id INTEGER, product_id INTEGER, quantity INTEGER);")

cursor.executemany("INSERT INTO Orders VALUES (?, ?, ?);", [
    (1, '2024-01-02', 1),
    (2, '2024-01-05', 2)
])

cursor.executemany("INSERT INTO Customers VALUES (?, ?);", [
    (1, 'Alice'),
    (2, 'Bob')
])

cursor.executemany("INSERT INTO Order_Details VALUES (?, ?, ?);", [
    (1, 101, 2),
    (1, 102, 1),
    (2, 101, 3)
])

cursor.execute("""
SELECT
    o.order_id,
    c.customer_name,
    od.quantity
FROM Orders o
INNER JOIN Customers c ON o.customer_id = c.customer_id
INNER JOIN Order_Details od ON o.order_id = od.order_id;
""")

for row in cursor.fetchall():
    print(row)


(1, 'Alice', 1)
(1, 'Alice', 2)
(2, 'Bob', 3)
